In [1]:
from src.value import Value
import src.function as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = Value(X_train, "x_train", requires_grad=False)
X_test = Value(X_test, "x_test", requires_grad=False)
y_train = Value(y_train, "y_train", requires_grad=False)
y_test = Value(y_test, "y_test", requires_grad=False)

In [3]:
w = Value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = Value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F._FunctionFactory().get_new_function_of_type(F._Matmul)
add1 = F._FunctionFactory().get_new_function_of_type(F._Add)
loss = F._FunctionFactory().get_new_function_of_type(F._BCELossWithLogits)
    
def fit(X, num_epoch):     
    for i in range(num_epoch):
        
        y = add1.forward(matmul1.forward(X, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.value -= lr * w.grad
        b.value -= lr * b.grad
        l.zero_grad()   
        
        # if i % 10:
        print(f"Epoch {i+1}/{num_epoch}: Train Loss: {l.value}")

def predict(X):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = F.sigmoid(y).value > 0.5
    
    return y_pred

Creating Function of type <class 'src.function._Matmul'> with name matmul_0
Creating Function of type <class 'src.function._Add'> with name add_0
Creating Function of type <class 'src.function._BCELossWithLogits'> with name bce_loss_logit_0


In [5]:
fit(X_train, 100)    

Epoch 1/100: Train Loss: 0.7111910626456543
Epoch 2/100: Train Loss: 0.35819554342383836
Epoch 3/100: Train Loss: 0.23052526520194486
Epoch 4/100: Train Loss: 0.16445779775753966
Epoch 5/100: Train Loss: 0.13177811910804282
Epoch 6/100: Train Loss: 0.11847836847010143
Epoch 7/100: Train Loss: 0.1101922716558072
Epoch 8/100: Train Loss: 0.10408022589488661
Epoch 9/100: Train Loss: 0.09926509683589126
Epoch 10/100: Train Loss: 0.09530701821851074
Epoch 11/100: Train Loss: 0.09195705785402104
Epoch 12/100: Train Loss: 0.08907265196882662
Epoch 13/100: Train Loss: 0.08656838335847117
Epoch 14/100: Train Loss: 0.08438325298593806
Epoch 15/100: Train Loss: 0.0824622599369914
Epoch 16/100: Train Loss: 0.08075315288489458
Epoch 17/100: Train Loss: 0.07921198386660316
Epoch 18/100: Train Loss: 0.0778068756672075
Epoch 19/100: Train Loss: 0.07651659784544076
Epoch 20/100: Train Loss: 0.0753270236815666
Epoch 21/100: Train Loss: 0.07422818779497128
Epoch 22/100: Train Loss: 0.0732125055079567
Epo

In [6]:
y_train_pred = predict(X_train)
y_test_pred = predict(X_test)

Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_0
Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_1


In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.value, y_train_pred)
print("")
print_metrics(y_test.value, y_test_pred)

Confusion Matrix
[[160   4]
 [  0 262]]

Accuracy: 0.9906103286384976
Precision: 0.9849624060150376
Recall: 1.0
AUC score: 0.9878048780487805
MCC score: 0.9802749270983103
F1-Score: 0.9924242424242424


Confusion Matrix
[[47  1]
 [ 1 94]]

Accuracy: 0.986013986013986
Precision: 0.9894736842105263
Recall: 0.9894736842105263
AUC score: 0.9843201754385964
MCC score: 0.968640350877193
F1-Score: 0.9894736842105263

